In [119]:
import pandas as pd
import numpy as np
import selection
import weights
import btag
import root_pandas
import importlib
import jetmet
import test_weights

In [2]:
data = ['Run2011A_MultiJet', 'Run2011B_MultiJet']
mc = ['T_TuneZ2_s', 'WJetsToLNu', 'DYJetsToLL', 'T_TuneZ2_tW', 'T_TuneZ2_t-channel',
       'Tbar_TuneZ2_s', 'Tbar_TuneZ2_tW', 'Tbar_TuneZ2_t-channel', 'TTJets']

In [2]:
import ROOT
#f = ROOT.TFile()

In [8]:
from coffea.lookup_tools import extractor

In [9]:
ext = extractor()
ext.add_weight_sets([
    "* * data/Legacy11_V1_DATA_UncertaintySources_AK5PFchs.junc.txt",
    "* * jer_sf.root",
    "* * JESUncMC.root"
])
ext.finalize()

evaluator = ext.make_evaluator()

In [156]:
ext = extractor()
ext.add_weight_sets([
    "jet40_ * rootFilesTurnOn/TriggerEffHisto_data_match40_JETLEG.root",
    "jet45_ * rootFilesTurnOn/TriggerEffHisto_data_match45_JETLEG.root",
    "tau40_ * rootFilesTurnOn/TriggerEffHisto_match40_newTauID.root",
    "tau45_ * rootFilesTurnOn/TriggerEffHisto_match45_newTauID.root"
])
ext.finalize()

evaluator = ext.make_evaluator()

In [157]:
evaluator.keys()

dict_keys(['jet40_jet1_ref', 'jet40_jet1_ref_error', 'jet40_jet2_ref', 'jet40_jet2_ref_error', 'jet40_jet3_ref', 'jet40_jet3_ref_error', 'jet40_jet4_ref', 'jet40_jet4_ref_error', 'jet40_jet1_trig', 'jet40_jet1_trig_error', 'jet40_jet2_trig', 'jet40_jet2_trig_error', 'jet40_jet3_trig', 'jet40_jet3_trig_error', 'jet40_jet4_trig', 'jet40_jet4_trig_error', 'jet40_jet1_eff', 'jet40_jet1_eff_error', 'jet40_jet2_eff', 'jet40_jet2_eff_error', 'jet40_jet3_eff', 'jet40_jet3_eff_error', 'jet40_jet4_eff', 'jet40_jet4_eff_error', 'jet40_reference_tau', 'jet40_reference_tau_error', 'jet40_triggered_tau', 'jet40_triggered_tau_error', 'jet40_eff_tau', 'jet40_eff_tau_error', 'jet40_reference_tau_eta', 'jet40_reference_tau_eta_error', 'jet40_triggered_tau_eta', 'jet40_triggered_tau_eta_error', 'jet40_eff_tau_eta', 'jet40_eff_tau_eta_error', 'jet40_HLTtau_pt', 'jet40_HLTtau_pt_error', 'jet40_HLTtau_eta', 'jet40_HLTtau_eta_error', 'jet40_HLTtau_pt_diff', 'jet40_HLTtau_pt_diff_error', 'jet40_HLTtau_eta_dif

In [165]:
def eval_weight(obj, key):
    
    weight = evaluator[key](obj)
    mask = weight == 0.
    return np.where(mask, 1., weight)

In [172]:
def calc_weight(jet, tau, prefix):
    
    jet0 = eval_weight(jet.pt[:,0], "jet" + prefix + "_jet4_eff" )
    jet1 = eval_weight(jet.pt[:,1], "jet" + prefix + "_jet4_eff" )
    jet2 = eval_weight(jet.pt[:,2], "jet" + prefix + "_jet4_eff" )
    tau0 = eval_weight(tau.pt[:,0], "tau" + prefix + "_eff_tau" )
    
    jet0_err = eval_weight(jet.pt[:,0], "jet" + prefix + "_jet4_eff_error" )
    jet1_err = eval_weight(jet.pt[:,1], "jet" + prefix + "_jet4_eff_error" )
    jet2_err = eval_weight(jet.pt[:,2], "jet" + prefix + "_jet4_eff_error" )
    tau0_err = eval_weight(tau.pt[:,0], "tau" + prefix + "_eff_tau_error" )
    
    weight = 1. * jet0 * jet1 * jet2 * tau0
    weight_up = 1. * (jet0 + jet0_err) * (jet1 + jet1_err) * (jet2 + jet2_err) * (tau0 + tau0_err)
    weight_down = 1. * (jet0 - jet0_err) * (jet1 - jet1_err) * (jet2 - jet2_err) * (tau0 - tau0_err)
    
    return weight, weight_up, weight_down

In [178]:
np.random.seed(0)

def trigger_weight(jet, tau, frac=0.218):
    
    # Randomly select 40/45 trigger according to lumi fraction
    rand = np.random.random((len(jet)))
    mask_is40 = rand < frac
    
    weight_40, weight_up_40, weight_down_40 = calc_weight(jet, tau, "40")
    weight_45, weight_up_45, weight_down_45 = calc_weight(jet, tau, "45")
    
    weight = np.where(mask_is40, weight_40, weight_45)
    weight_up = np.where(mask_is40, weight_up_40, weight_up_45)
    weight_down = np.where(mask_is40, weight_down_45, weight_down_45)
    
    return weight, weight_up, weight_down
    

In [179]:
weight = trigger_weight(jet, tau)

In [3]:
f = ROOT.TFile("data/beff.root")
h2_BTaggingEff_Denom_b = f.Get("h2_BTaggingEff_Denom_b")
h2_BTaggingEff_Num_b = f.Get("h2_BTaggingEff_Num_b")
h2_BTaggingEff_Denom_c = f.Get("h2_BTaggingEff_Denom_c")
h2_BTaggingEff_Num_c = f.Get("h2_BTaggingEff_Num_c")
h2_BTaggingEff_Denom_usdg = f.Get("h2_BTaggingEff_Denom_udsg")
h2_BTaggingEff_Num_usdg = f.Get("h2_BTaggingEff_Num_udsg")

In [4]:
def to_eff(h1, h2):
    
    h3 = h1.Clone("h3")
    h3.Sumw2()
    h3.Divide(h2)
    return h3

In [10]:
outfile = ROOT.TFile("data/beff_precalc.root", "RECREATE")
b = to_eff(h2_BTaggingEff_Num_b, h2_BTaggingEff_Denom_b)
c = to_eff(h2_BTaggingEff_Num_c, h2_BTaggingEff_Denom_c)
usdg = to_eff(h2_BTaggingEff_Num_usdg, h2_BTaggingEff_Denom_usdg)
b.Write("b")
c.Write("c")
usdg.Write("usdg")
outfile.Close()

In [11]:
ff = ROOT.TFile("data/beff_precalc.root")

In [12]:
ext = extractor()
ext.add_weight_sets([
    "* * data/beff_precalc.root"
])
ext.finalize()

evaluator = ext.make_evaluator()

In [13]:
evaluator.keys()

dict_keys(['b', 'b_error', 'c', 'c_error', 'usdg', 'usdg_error'])

In [14]:
import root_numpy

In [15]:
h1 = ff.Get("b")
h2 = ff.Get("c")
h3 = ff.Get("usdg")

In [17]:
b_num, edges = root_numpy.hist2array(h1, return_edges=True)
c_num, _ = root_numpy.hist2array(h2, return_edges=True)
usdg_num, _ = root_numpy.hist2array(h3, return_edges=True)

In [18]:
c_num.shape

(100, 60)

In [134]:
sf = np.dstack((usdg_num, c_num, b_num ))

In [135]:
sf.shape

(100, 60, 3)

In [136]:
sf[jet.pt, jet.eta, 0]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [137]:
hist = ROOT.TH3F("hist", "hist", 100, 0., 1000., 60, -3., 3., 3, -0.5, 2.5)

In [138]:
from root_numpy import array2hist, hist2array

In [139]:
_ = array2hist(sf, hist)

In [140]:
outtest = ROOT.TFile("data/beff_test.root", "RECREATE")
hist.Write("eff")
outtest.Close()

In [141]:
fff = ROOT.TFile("data/beff_test.root")

In [142]:
fff.ls()

TFile**		data/beff_test.root	
 TFile*		data/beff_test.root	
  KEY: TH3F	eff;1	hist


In [143]:
ext = extractor()
ext.add_weight_sets([
    "* * data/beff_test.root"
])
ext.finalize()

evaluator = ext.make_evaluator()

In [144]:
evaluator.keys()

dict_keys(['eff', 'eff_error'])

In [145]:
evaluator["eff"]

3 dimensional histogram with axes:
	1: [   0.   10.   20.   30.   40.   50.   60.   70.   80.   90.  100.  110.
  120.  130.  140.  150.  160.  170.  180.  190.  200.  210.  220.  230.
  240.  250.  260.  270.  280.  290.  300.  310.  320.  330.  340.  350.
  360.  370.  380.  390.  400.  410.  420.  430.  440.  450.  460.  470.
  480.  490.  500.  510.  520.  530.  540.  550.  560.  570.  580.  590.
  600.  610.  620.  630.  640.  650.  660.  670.  680.  690.  700.  710.
  720.  730.  740.  750.  760.  770.  780.  790.  800.  810.  820.  830.
  840.  850.  860.  870.  880.  890.  900.  910.  920.  930.  940.  950.
  960.  970.  980.  990. 1000.]
	2: [-3.  -2.9 -2.8 -2.7 -2.6 -2.5 -2.4 -2.3 -2.2 -2.1 -2.  -1.9 -1.8 -1.7
 -1.6 -1.5 -1.4 -1.3 -1.2 -1.1 -1.  -0.9 -0.8 -0.7 -0.6 -0.5 -0.4 -0.3
 -0.2 -0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1
  1.2  1.3  1.4  1.5  1.6  1.7  1.8  1.9  2.   2.1  2.2  2.3  2.4  2.5
  2.6  2.7  2.8  2.9  3. ]
	3: [-0.5  0.5  1.5  2.5]

In [44]:
abs(jet["flavour"])

<JaggedArray [[5 2 3 5] [21 21 1 5 21 5] [1 5 5 5 21] ... [21 5 5 21] [5 5 4 21] [5 1 5 21]] at 0x7f3b15cb7550>

In [45]:
from coffea.btag_tools import BTagScaleFactor

In [47]:
btag_sf = BTagScaleFactor("data/CSV.csv", "medium", methods='comb,comb,comb')


In [65]:
btag_sf.eval("central", np.array(5), np.array(0.), np.array(100.))

array(0.96321061)

In [76]:
light_mask = (abs(jet["flavour"].content) != 4) & (abs(jet["flavour"].content) != 5)

In [77]:
abs(jet["flavour"])

<JaggedArray [[5 2 3 5] [21 21 1 5 21 5] [1 5 5 5 21] ... [21 5 5 21] [5 5 4 21] [5 1 5 21]] at 0x7f3b158bef90>

In [78]:
light_mask

array([False,  True, False, ...,  True,  True,  True])

In [74]:
light_id = np.zeros_like(jet.flavour.content)

In [80]:
flav = jet.flavour.content

In [81]:
flav

array([-5, 21,  5, ...,  2,  1, -1], dtype=int32)

In [82]:
jsmear_cen = np.where(light_mask, light_id, flav)
                      

#1 + (jersf_cent.content - 1.) * (jet.pt.content - ptGenJet) / jet.pt.content,
#1. + np.sqrt(np.maximum(jersf_cent.content**2 - 1.0, 0)) * jersmear)


In [83]:
jsmear_cen

array([-5,  0,  5, ...,  0,  0,  0], dtype=int32)

In [84]:
jet.add_attributes(jsmear_cen = jsmear_cen)

ValueError: starts and stops are not compatible with a single offsets array

In [93]:
jet._content._contents['test'] = jsmear_cen

In [95]:
jet["test"]

IndexError: index 1069 is out of bounds for axis 0 with size 1069

In [92]:
jet["pt"]

<JaggedArray [[163.20390936847798 147.0705849695005 87.1744582996871 49.84018716016726] [226.90783273172656 120.50872391408848 54.467506913662874 50.84569503417879 45.31622754021464 44.78825205443172] [299.1122238112803 199.61652207597945 73.74366771523637 71.4092503013294 46.73008492162808] ... [243.62729177157235 130.0149907242394 100.40854891189656 55.45967043634846] [130.9353703983288 95.68388805153063 77.29572579462365 48.115007893602524] [105.81756811535615 69.69033395377892 53.68807089895199 37.05074228692138]] at 0x7f3b158c43d0>

In [265]:
evaluator["eff"](100., 0.15, 2.)

0.012711762

In [211]:
import copy

In [227]:
tau["p4"][:,0].Et

array([ 51.16976 ,  45.56891 ,  67.140274,  48.31369 ,  76.19487 ,
        56.50825 ,  74.51507 , 131.81163 ,  76.65389 ,  46.722996,
        61.082962,  87.23273 ,  51.18036 ,  84.497215,  46.508938,
        47.541   ,  49.130108,  52.47772 ,  71.87377 ,  86.26697 ,
        77.63009 ,  47.035297,  67.27164 ,  87.76323 ,  51.415882,
        91.65241 ,  94.6464  ,  52.640915,  57.595932,  64.01166 ],
      dtype=float32)

In [232]:
jet["p4"].sum().E + tau["p4"][:,0].E

array([ 571.6840223 , 1143.70263615, 1345.4132791 ,  435.71137959,
       1386.39694805,  389.05228978,  456.52148583,  797.66127834,
       1310.56745499,  452.65022203,  468.88114205,  949.72144692,
       1561.29354803,  599.21622804,  414.15598934,  650.1362376 ,
        847.37473553,  743.3744436 ,  713.05591442,  742.09827882,
        637.17812492,  638.85911981,  487.81153397,  595.52384529,
        485.51872162,  874.18085023, 1178.84257177,  779.53703789,
        552.3559195 ,  540.72651158])

In [243]:
jet["p4"].sum().E

array([ 511.21901452,  916.90206852, 1243.37925963,  366.49295765,
       1293.93013592,  324.49452   ,  380.1640258 ,  657.9206625 ,
       1233.00531358,  402.64416047,  405.05368844,  642.13578407,
       1483.82667487,  416.47126344,  367.44997279,  547.25395915,
        676.85943096,  552.09078271,  617.40896465,  654.98034364,
        558.40590812,  590.13784067,  386.56255631,  481.36860237,
        428.28467087,  763.58439943,  961.83839086,  685.33297447,
        492.21066727,  465.39887487])

In [244]:
#for j in jet:
H_jet = 0.
for vec in jet[0]:
    H_jet += vec["p4"].E
    print( vec["p4"].E )
print(H_jet)
print(tau[0]["p4"].E)

164.01991299022782
172.19700967785408
104.7502209333932
70.25186421534829
511.21900781682336
[60.465008]


In [239]:
jet[0]["p4"].E

array([164.01991375, 172.19701706, 104.75021976,  70.25186395])

In [246]:
for j, t in zip(jet, tau):
    
    p4 = copy.deepcopy(t["p4"][0])
    #print("Tau p4", p4)
    H = t["p4"][0].E
    HT = t["p4"][0].Et
    H_jet, HT_jet = 0., 0.
    for vec in j:
        H += vec["p4"].E
        H_jet += vec["p4"].E 
        HT += vec["p4"].Et
        p4 += vec["p4"]
    print(H, HT)
    #print("Added", p4)
    #print( j["p4"].pt, t["p4"].pt )

571.6840143202667 501.4935059754949
1143.702637230435 594.1140530593544
1345.4132401197817 760.2089534326599
435.7113842632429 329.02065909621405
1386.3968871233856 499.05931233994403
389.0522886169456 287.98253435152856
456.5214762305045 330.6654799448787
797.6612820675463 678.0166004185505
1310.5674491113102 963.444294702688
452.6502244298722 323.220225196699
468.88115448975327 393.1921865292365
949.721443479174 499.21046518509866
1561.2935456223472 513.6106187798846
599.2162344811418 329.6534370986973
414.1559870961348 358.63848351616406
650.1362291060015 481.6300053227463
847.3747438222822 369.6056072179997
743.3744463598224 392.2333782313991
713.0559216994559 419.61288214236816
742.0982738095427 578.5526807530725
637.1781118379565 422.0779466349294
638.8591170250485 407.591209829311
487.81154104118787 335.0084475875042
595.5238484936078 457.72256516130216
485.51872160800247 285.3855625489791
874.180852640075 400.4859585535469
1178.8425621799734 580.4924236227673
779.5370359583246 

In [205]:
jet["p4"][0]

<PtEtaPhiMassLorentzVectorArray [TLorentzVector(163.2, -0.019414, 0.40543, 16.03) TLorentzVector(147.07, -0.5662, -2.1255, 17.739) TLorentzVector(87.174, 0.60444, -3.1305, 15.547) TLorentzVector(49.84, -0.86283, -1.2753, 9.7447)] at 0x7fc9d4bb41d0>

In [206]:
sum(jet["p4"][0])

TypeError: cannot add a TLorentzVector with a int

In [203]:
evaluator["usdg"](jet.pt, jet.eta)

ValueError: starts and stops are not compatible with a single offsets array

In [31]:
evaluator.keys()

dict_keys(['jet40', 'jet45', 'tau40', 'tau45'])

In [32]:
evaluator["jet40"](100.)

0.9672131

In [99]:
jet.columns

['p4',
 '__fast_pt',
 '__fast_eta',
 '__fast_phi',
 '__fast_mass',
 'pt_jer_up',
 'mass_jer_up',
 'pt_jer_down',
 'mass_jer_down',
 'pt_jes_up',
 'mass_jes_up',
 'pt_jes_down',
 'mass_jes_down',
 'pt_jes_up_old',
 'mass_jes_up_old',
 'pt_jes_down_old',
 'mass_jes_down_old',
 'jes_up',
 'jes_down',
 'jes_up_old',
 'jes_down_old',
 'csvDisc',
 'flavour',
 'pt',
 'px',
 'py',
 'pz',
 'e',
 'eta',
 'mass',
 'phi',
 'test']

In [208]:
importlib.reload(selection)
importlib.reload(jetmet)
importlib.reload(test_weights)
path = "/eos/user/l/llayer/opendata_files/preselection_merged/" + "T_TuneZ2_s" + ".root"
df, jet, tau, cut_flow = selection.event_selection(path, isData = False, isTT = False, corrLevel = "centJER")

Processing: T_TuneZ2_s.root isData: False isTT: False corrLevel centJER
SF 1.0 1.0
Test btag
[0.7724293  0.5934119  0.6328711  0.707928   0.71893275 0.61599267
 0.52717245 0.5961134  0.66737014 0.567685   0.67754585 0.77963734
 0.78650624 0.65044487 0.6543218  0.6944726  0.6269271  0.68879235
 0.7110556  0.7725621  0.6688743  0.6525197  0.6404202  0.67880666
 0.5049475  0.58545065 0.70343727 0.7268239  0.7198145  0.5910853 ]
(30, 6) (30, 3)


In [186]:
df.head()

,Jet_pt,Jet_px,Jet_py,Jet_pz,Jet_e,Jet_eta,Jet_phi,Jet_mass,Jet_csvDisc,Jet_flavour,...,MET_met,event,run,luminosityBlock,HLT_QuadJet40_IsoPFTau40,HLT_QuadJet45_IsoPFTau45,PV_npvs,trigger_weight,trigger_weight_up,trigger_weight_down
0,"[163.20390936847798, 147.0705849695005, 87.174...","[149.97339549338534, -77.45902065538972, -87.1...","[64.36999550980845, -125.01942427723347, -0.96...","[-3.1687033794641106, -87.79175116520194, 55.9...","[164.01991375479608, 172.1970170581061, 104.75...","[-0.01941439, -0.5661959, 0.60444456, -0.8628344]","[0.40543064, -2.1254857, -3.1304731, -1.2753406]","[16.03044426947463, 17.738706012353532, 15.547...","[0.99315065, 0.10967823, 0.044752024, 0.9723706]","[-5, -2, -3, 5]",...,50.940367,145002,1,484,1,1,14,0.772429,0.791844,0.753337
1,"[226.90783273172656, 120.50872391408848, 54.46...","[181.12231133575276, -49.77201212847478, -47.3...","[-136.68163805137937, 109.75017081313649, 26.8...","[74.6777780591364, 83.46608186486387, 124.1151...","[242.98094449039405, 147.36636631641025, 135.9...","[0.3234416, 0.6466056, 1.561753, 2.343851, 0.5...","[-0.6464623, 1.9965594, 2.6250632, 1.9775524, ...","[44.44995227179993, 15.096519567917085, 10.334...","[0.0010465897, 0.031641357, 0.35334906, 0.4049...","[21, 21, -1, 5, 21, -5]",...,38.528390,153527,1,512,1,1,6,0.593412,0.628852,0.558737
2,"[299.1122238112803, 199.61652207597945, 73.743...","[-293.455604359588, 63.13224027288757, 60.5878...","[-57.895904794517676, 189.37021191449261, -42....","[396.6279716156869, -343.6932015132869, -52.94...","[497.5497239899049, 399.83385726471613, 91.441...","[1.0942149, -1.3118047, -0.66732067, 0.895451,...","[-2.946804, 1.2490038, -0.60658246, -1.1269907...","[27.8180803401483, 43.535435542754385, 10.9660...","[0.104415126, 0.9577689, 0.3878398, 0.9601148,...","[1, -5, -5, 5, 21]",...,115.740209,200122,1,668,1,1,5,0.632871,0.697097,0.572092
3,"[100.37066608557325, 82.17995731680276, 55.564...","[-71.99163094691605, 44.77474981539156, -33.67...","[69.93908712284136, -68.9113004145677, -44.194...","[52.68486966276081, 74.46336014367235, -78.985...","[113.86825468269318, 111.14647583022689, 96.93...","[0.5033741, 0.813395, -1.150424, -0.36393678]","[2.370655, -0.9946006, -2.2219646, 0.36061528]","[10.77099484369569, 7.429770760218332, 8.41848...","[0.1243552, 0.9985866, 0.07168705, 0.15791535]","[21, 5, 1, 4]",...,12.859408,85960,1,287,1,1,5,0.707928,0.741848,0.614263
4,"[140.05608694511466, 111.24469770364976, 71.43...","[92.93188175237519, -111.15314761922609, 8.827...","[104.78250105281724, 4.512217098936061, -70.88...","[-663.8304245719105, -299.62831495182826, -102...","[678.6373787673799, 319.9367048599929, 125.802...","[-2.260078, -1.7167629, -1.1578946, -1.4653825...","[0.8452646, 3.1010203, -1.4469057, -0.40418428...","[16.190442579586488, 14.386427332318704, 14.88...","[0.14651686, 0.9617057, 0.11498507, 0.91413635...","[2, -5, 21, 5, 2]",...,13.085134,246753,1,823,1,1,4,0.718933,0.734722,0.703383


In [209]:
jet.columns

['p4',
 '__fast_pt',
 '__fast_eta',
 '__fast_phi',
 '__fast_mass',
 'pt_jer_up',
 'mass_jer_up',
 'pt_jer_down',
 'mass_jer_down',
 'pt_jes_up',
 'mass_jes_up',
 'pt_jes_down',
 'mass_jes_down',
 'pt_jes_up_old',
 'mass_jes_up_old',
 'pt_jes_down_old',
 'mass_jes_down_old',
 'jes_up',
 'jes_down',
 'jes_up_old',
 'jes_down_old',
 'csvDisc',
 'flavour',
 'pt',
 'px',
 'py',
 'pz',
 'e',
 'eta',
 'mass',
 'phi',
 'flavour_sf',
 'flavour_eff',
 'b_eff',
 'b_sf',
 'btag_pmc',
 'btag_pdata',
 'btag_pdata_up',
 'btag_pdata_down']

In [197]:
def btag_weight_1(jet):
    
    pmc = jet.btag_pmc.prod()
    pdata = jet.btag_pdata.prod()
    
    return pdata/pmc

In [198]:
btag_weight_1(jet)

array([0.9234422 , 0.99195748, 1.04076518, 0.97353385, 0.9253162 ,
       0.95846693, 0.93465915, 1.16799741, 0.93280043, 0.93519822,
       0.96024376, 0.93280285, 1.01798248, 0.97548539, 0.91901519,
       0.92636441, 0.95994961, 0.95482549, 0.96363029, 1.02222381,
       1.02939833, 1.06086587, 1.03926747, 1.03886583, 1.03898086,
       0.92389271, 0.93309796, 0.92851544, 0.93931945, 1.04297376])

In [204]:
def btag_weight_2(jet):
    
    weight_0tags = (1 - jet.b_eff * jet.b_sf).prod()
    #print(weight_0tags)
    return 1. - weight_0tags

In [205]:
btag_weight_2(jet)

array([0.90234404, 0.81017315, 0.9707715 , 0.78195841, 0.87002757,
       0.64389106, 0.91396414, 0.87951317, 0.89892989, 0.9318225 ,
       0.72149797, 0.89673109, 0.86915164, 0.72875756, 0.90839227,
       0.92804666, 0.64219107, 0.67141058, 0.72541953, 0.93320702,
       0.88764083, 0.9034709 , 0.90415145, 0.89185172, 0.89627766,
       0.89297223, 0.87782794, 0.91752792, 0.93875455, 0.89661354])

In [206]:
(jet.csvDisc > 0.679).sum()

array([2, 1, 2, 1, 2, 1, 2, 0, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1,
       1, 2, 1, 2, 2, 2, 2, 1])

In [200]:
jet["flavour_sf"]

<JaggedArray [[5 0 0 5] [0 0 0 5 0 5] [0 5 5 5 0] ... [0 5 5 0] [5 5 4 0] [5 0 5 0]] at 0x7f3b1470bcd0>

In [149]:
jet["flavour_eff"]

<JaggedArray [[2 0 0 2] [0 0 0 2 0 2] [0 2 2 2 0] ... [0 2 2 0] [2 2 1 0] [2 0 2 0]] at 0x7f3b152c9ad0>

In [162]:
jet["sb_eff"]

<JaggedArray [[0.73355883 0.01414611 0.013459794 0.6844905] [0.013669968 0.0127615575 0.01442174 0.4758115 0.014091027 0.6604813] [0.015353122 0.6464147 0.75262946 0.73776287 0.019866053] ... [0.011759935 0.7298863 0.740727 0.014267276] [0.7298863 0.7629098 0.21641394 0.015026693] [0.740727 0.013364381 0.65340453 0.024175556]] at 0x7f3b16017c50>

In [163]:
jet["b_sf"]

<JaggedArray [[0.9660021032534176 1.058502905224619 1.0169255390833172 0.956966395329308] [1.0895008326669198 1.0421992674442098 0.9633286386156684 0.9571887021974084 0.9769345485778116 0.9557313605668698] [1.0423010672027793 0.9668550397625865 0.960848008097019 0.9605663762820974 0.9006504091112063] ... [1.0929843174881417 0.9648449914211386 0.9632386873900091 0.9875127716376121] [0.9648842990021972 0.9629010010873107 0.9612484470092408 0.9799116609402363] [0.9635921744359831 1.001384110948794 0.9577805852487337 0.8949867175660247]] at 0x7f3b147f2c10>

In [187]:
import awkward as ak

In [190]:
(jet["b_sf"]).prod()

array([0.99507535, 0.97757857, 0.83770841, 0.90980624, 0.82354339,
       0.89036902, 0.86601121, 0.90118273, 0.91353863, 0.85444292,
       0.97043742, 0.90372086, 0.86435666, 0.8718788 , 0.92678827,
       0.97828252, 0.87222042, 0.95825176, 0.83714541, 0.84236062,
       0.84461811, 0.84910529, 0.87732569, 0.9131815 , 0.75887621,
       0.72458494, 0.91507439, 1.00310897, 0.87514387, 0.82713535])

In [117]:
jet.pt[:,0]

array([163.20390937, 226.90783273, 299.11222381, 100.37066609,
       140.05608695,  78.91916171,  90.09379455, 341.93844147,
       448.63667939, 118.83695845, 118.98076151, 170.23100474,
       189.56496964,  98.71316179, 108.18647825, 194.11182327,
       173.88340721, 151.78812717, 144.87926422, 169.836751  ,
        99.5502518 , 117.42401284,  97.75288404, 178.08219253,
        62.58631411, 110.52928477, 213.08795233, 243.62729177,
       130.9353704 , 105.81756812])

In [80]:
jet40 = evaluator["jet40"](jet.pt[:,0])

In [81]:
jet40

array([1.        , 0.        , 0.        , 0.94067794, 0.9166667 ,
       0.90782124, 0.93877554, 0.        , 0.        , 0.93333334,
       0.93333334, 1.        , 1.        , 0.94067794, 0.9672131 ,
       0.        , 1.        , 1.        , 1.        , 1.        ,
       0.9672131 , 0.93333334, 0.94067794, 1.        , 0.89365673,
       0.93333334, 0.        , 0.        , 0.9166667 , 0.9672131 ],
      dtype=float32)

In [82]:
mask = jet40 == 0.

In [83]:
mask

array([False,  True,  True, False, False, False, False,  True,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False,  True,
        True, False, False])

In [87]:
jet40 = np.where(mask, 1., jet40)

In [88]:
jet40

array([1.        , 1.        , 1.        , 0.94067794, 0.9166667 ,
       0.90782124, 0.93877554, 1.        , 1.        , 0.93333334,
       0.93333334, 1.        , 1.        , 0.94067794, 0.9672131 ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.9672131 , 0.93333334, 0.94067794, 1.        , 0.89365673,
       0.93333334, 1.        , 1.        , 0.9166667 , 0.9672131 ],
      dtype=float32)

In [ ]:
def trigger_weight(jet, tau):
    
    

In [42]:
t = np.random.random((len(jet)))

In [45]:
t < 0.218

array([ True, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False,  True, False,
        True, False, False])

In [49]:
tt = np.where(t < 0.218, 0.7, 0.3)

In [50]:
tt

array([0.7, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.7, 0.3, 0.3, 0.3, 0.3,
       0.3, 0.3, 0.3, 0.3, 0.3, 0.7, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.7,
       0.3, 0.7, 0.3, 0.3])

In [ ]:
tt = np.where(t < 0.218, 0.7, 0.3)

In [11]:
importlib.reload(selection)
def event_selection():
    
    samples = {}
    for sample in mc + data:
        
        #!!!!!!! Careful with JER application before Tau?!
        
        if "TTJets" in sample: isTT = True
        else: isTT = False
            
        if "Run2011" in sample: isData = True
        else: isData = False
        
        path = "/eos/user/l/llayer/opendata_files/preselection_merged/" + sample + ".root"
        
        df, cut_flow = selection.event_selection(path, isData = isData, isTT = isTT)
        
        print( cut_flow )
        """
        df.to_hdf(outpath + sample + ".h5", "central", mode='w')
                
        if isData == False:
            
            for c in corrections:
                df, cut_flow = selection.event_selection(path, isData = isData, isTT = isTT, corrLevel = c)
                print( cut_flow )
                df.to_hdf(outpath + sample + ".h5", c, mode='a')  
        """
        
        samples[sample] = df
        
    return samples

samples = event_selection()

Processing: T_TuneZ2_s.root isData: False isTT: False corrLevel cent
{'preselected': 1915, 'hlt': 273, 'lep_veto': 190, 'jet_requirement': 114, 'tau_requirement': 30}
Processing: WJetsToLNu.root isData: False isTT: False corrLevel cent
{'preselected': 44906, 'hlt': 4982, 'lep_veto': 2385, 'jet_requirement': 1083, 'tau_requirement': 231}
Processing: DYJetsToLL.root isData: False isTT: False corrLevel cent
{'preselected': 114807, 'hlt': 19961, 'lep_veto': 5461, 'jet_requirement': 2265, 'tau_requirement': 547}
Processing: T_TuneZ2_tW.root isData: False isTT: False corrLevel cent
{'preselected': 20110, 'hlt': 5529, 'lep_veto': 3046, 'jet_requirement': 1698, 'tau_requirement': 568}
Processing: T_TuneZ2_t-channel.root isData: False isTT: False corrLevel cent
{'preselected': 23527, 'hlt': 3396, 'lep_veto': 2404, 'jet_requirement': 1203, 'tau_requirement': 240}
Processing: Tbar_TuneZ2_s.root isData: False isTT: False corrLevel cent
{'preselected': 1117, 'hlt': 158, 'lep_veto': 115, 'jet_requir

In [12]:
importlib.reload(btag)
importlib.reload(weights)

def candidates(sample, df, invert_btag = False, njets=-1):
    
    if "Run2011" in sample: isData = True
    else: isData = False

    print( "Processing:", sample,"isData:", isData, "invert_btag:", invert_btag)

    df['nJets'] = df["Jet_pt"].str.len()
    
    # b-tagging
    df["Jet_nbtags"] = df["Jet_csvDisc"].apply( lambda x : btag.count_btags(x, njets=njets) )
    if invert_btag:
        df = btag.no_tag(df)
    else:
        df = btag.at_least_1tag(df)

    # MET cut
    df = selection.met_requirement(df)

    
    # HL features
    #df = pd.concat([df, df.apply(lambda ev : pd.Series(hl.hlFeatures(ev, njets=njets)), axis=1)], axis=1)
    

    # MC weights
    if not isData:

        hlt_40, hlt_45 = weights.lumi()
        total_lumi = hlt_40 + hlt_45
        trigger_frac = hlt_40 / float(hlt_45)
        df = pd.concat([df, df.apply(lambda ev: pd.Series(btag.eval_sf_eff(ev)), axis=1)], axis=1)
        df["Jet_btag_weight1"] = df.apply(lambda ev : btag.b_weight_method1(ev, njets=njets), axis=1)
        #df["Jet_btag_weight2"] = df.apply(lambda ev : btag.b_weight_method2(ev, njets=njets), axis=1)
        # trigger weights
        #df["trigger_weight"] = df.apply(lambda ev : weights.trigger_weight(ev, trigger_frac), axis=1)
        df = pd.concat([df, df.apply(lambda ev: pd.Series(weights.trigger_weight(ev, trigger_frac)), axis=1)], axis=1)
        # normalization
        counts_path = "/eos/user/l/llayer/opendata_files/preselection_merged/" + sample + "_counts.root"
        total_counts = root_pandas.read_root(counts_path)
        xsec = weights.get_xsec(sample)
        weights.norm(df, total_counts, xsec, lumi = total_lumi)

    # QCD
    if isData & invert_btag:
        
        # Assume light flavour
        def lf(nJets):
            return np.zeros((nJets))
        df["Jet_flavour"] = df["nJets"].apply(lf)
        df = pd.concat([df, df.apply(lambda ev: pd.Series(btag.eval_sf_eff(ev)), axis=1)], axis=1)
        df["btag_weight"] = df.apply(lambda ev : btag.b_weight_method2(ev, njets=njets), axis=1)
        
    return df

/eos/home-l/llayer/cmsopen/columnar/btag.py:13: RuntimeWarning: invalid value encountered in true_divide
  eff = np.divide(num[0], denom[0])


In [13]:
def rearrange_samples(samples):
    
    # Concat and split MC in signal and background
    new_samples = {}
    new_samples["TTJets_signal"], new_samples["TTJets_bkg"] = weights.classify_tt(samples["TTJets"])
    new_samples["WZJets"] = pd.concat([samples['WJetsToLNu'], samples['DYJetsToLL']], axis=0)
    new_samples["STJets"] = pd.concat([samples['T_TuneZ2_s'], samples['T_TuneZ2_tW'], samples['T_TuneZ2_t-channel'], 
                              samples['Tbar_TuneZ2_s'], samples['Tbar_TuneZ2_tW'], 
                              samples['Tbar_TuneZ2_t-channel']], axis=0)

    # Concat the data
    new_samples["Data"] = pd.concat([samples["Run2011A_MultiJet"], samples["Run2011B_MultiJet"]], axis=0)
    new_samples["QCD"] = pd.concat([samples["QCD_Run2011A_MultiJet"], samples["QCD_Run2011B_MultiJet"]], axis=0)
    
    return new_samples

In [14]:
def proc_candidates(samples, njets = -1):
    
    cand_samples = {}
    for sample in data:
        cand_samples[sample] = candidates(sample, samples[sample], invert_btag = False, njets=njets)
        cand_samples["QCD_" + sample] = candidates(sample, samples[sample], invert_btag = True, njets=njets)
    
    
    for sample in mc:
        cand_samples[sample] = candidates(sample, samples[sample], invert_btag = False, njets=njets)   

    new_samples = rearrange_samples(cand_samples)
    return new_samples

In [15]:
cands = proc_candidates(samples)

Processing: Run2011A_MultiJet isData: True invert_btag: False
Processing: Run2011A_MultiJet isData: True invert_btag: True
Processing: Run2011B_MultiJet isData: True invert_btag: False
Processing: Run2011B_MultiJet isData: True invert_btag: True
Processing: T_TuneZ2_s isData: False invert_btag: False
Processing: WJetsToLNu isData: False invert_btag: False
Processing: DYJetsToLL isData: False invert_btag: False
Processing: T_TuneZ2_tW isData: False invert_btag: False
Processing: T_TuneZ2_t-channel isData: False invert_btag: False
Processing: Tbar_TuneZ2_s isData: False invert_btag: False
Processing: Tbar_TuneZ2_tW isData: False invert_btag: False
Processing: Tbar_TuneZ2_t-channel isData: False invert_btag: False
Processing: TTJets isData: False invert_btag: False


In [16]:
len(cands["Data"])

3238

In [18]:
import ROOT
from root_numpy import fill_hist

def save_var(sample, name, var_name, bins = 20, xlow = 0., xup = 350):

    hist = ROOT.TH1D(name + "_" + var_name, name + "_" + var_name, bins, xlow, xup)
    hist.Sumw2()
    
    if name == "Data":
        pass
    elif name == "QCD":
        if var_name == "bdt":
            scale_qcd = 9.
            sample = sample[sample["train_flag"] == "test"]
        else:
            scale_qcd = 4.3
        sample['weight'] = sample['btag_weight'] * scale_qcd
    else:
        #samples[sample]['new_trigger_weight'] = new_samples[sample].apply(lambda ev : weights.trigger_weight(ev), axis=1)
        sample['weight'] = sample['norm'] * (1/1000) * sample['trigger_weight'] * sample['Jet_btag_weight1']
        #print(sample, sum(samples[sample]['weight']))
        #new_samples[sample]['btag_weight2']
    
    # Flatten if the column is a list
    if "Jet_" in var_name:
        series = sample[var_name].apply(pd.Series).stack().reset_index(drop=True)
        if name != "Data":
            sample['weight_stacked'] = sample.apply(lambda x : stack_weight(x["weight"] ,x["nJets"]), axis=1)
            weights = sample['weight_stacked'].apply(pd.Series).stack().reset_index(drop=True)
    else:
        series = sample[var_name]
        if name != "Data":
            weights = sample["weight"]
    if name == "Data":
        fill_hist(hist, series)
    else:
        fill_hist(hist, series, weights = weights)
    #hist.Write()
    return hist

def vars_to_histos(samples, variables):
    
    hists = {}
    for name, sample in samples.items():
        for var in variables:
            hists[name + "_" + var["var_name"]] = save_var(sample, name, var["var_name"], var["bins"], var["xlow"], var["xup"])

    return hists

In [19]:
variables = [
    {"var_name" : "MET_met", "bins" : 30, "xlow" : 0., "xup" : 400, "xtitle" : "MET [GeV]"}
]

hists = vars_to_histos(cands, variables)

In [20]:
f = ROOT.TFile("histos_test.root", "RECREATE")
for s, hist in hists.items():

    print( s, hist.Integral() )
    hist.Write()
    
f.Close()

TTJets_signal_MET_met 345.9627057153264
TTJets_bkg_MET_met 145.5093453003518
WZJets_MET_met 61.43496068440135
STJets_MET_met 30.534209478997937
Data_MET_met 3238.0
QCD_MET_met 2731.115160766783


In [76]:
def fit(histos, fit_var, corr = "central"):
        

    bkg = histos['TTJets_bkg' + "_" + fit_var ].Clone()
    bkg.Add(histos['WZJets' + "_" + fit_var ])
    bkg.Add(histos['STJets' + "_" + fit_var ])

    data = histos["Data" + "_" + fit_var ].Clone()
    data.Add(bkg, -1.)

    signal = histos["TTJets_signal" + "_" + fit_var ]

    qcd = histos["QCD" + "_" + fit_var ]

    x = ROOT.RooRealVar("x","x",0.,350.)
    rooSignal =  ROOT.RooDataHist("signal","signal",ROOT.RooArgList( x ), signal)
    rooBkg = ROOT.RooDataHist("bkg","bkg", ROOT.RooArgList( x ), qcd)
    signal_pdf = ROOT.RooHistPdf("signal","signal",ROOT.RooArgSet( x ), rooSignal)
    bkg_pdf = ROOT.RooHistPdf("bkg","bkg",ROOT.RooArgSet( x ), rooBkg)

    c0 = ROOT.RooRealVar("c0","c0",0.5,0.,1.)
    pdf = ROOT.RooAddPdf("pdf","pdf", signal_pdf,bkg_pdf, c0)

    dataFit = ROOT.RooDataHist("data","data",ROOT.RooArgList( x ), data);

    fitResult = pdf.fitTo(dataFit)
    #print( fitResult )

    sf_tt_sig = (c0.getVal() * data.Integral()) / signal.Integral() 
    sf_tt_sig_err = (c0.getError() * data.Integral()) / signal.Integral()
    
    sf_qcd = ((1-c0.getVal())*data.Integral()) / qcd.Integral()
    sf_qcd_err = (c0.getError() * data.Integral()) / qcd.Integral()
    
    
    print( "scale factor TTbar tau(h) QQ ", sf_tt_sig, "+-", sf_tt_sig_err )
    print( "scale factor MultiJet ", sf_qcd, "+-", sf_qcd_err )

    return sf_tt_sig, sf_qcd

In [77]:
fit(hists, "MET_met")

scale factor TTbar tau(h) QQ  0.9910458961661615 +- 0.09467095587401728
scale factor MultiJet  0.97667607748668 +- 0.011718521532530633


(0.9910458961661615, 0.97667607748668)

[#1] INFO:DataHandling -- RooDataHist::adjustBinning(signal): fit range of variable x expanded to nearest bin boundaries: [0,350] --> [0,360]
[#0] WARNING:InputArguments -- RooAbsPdf::fitTo(pdf) WARNING: a likelihood fit is requested of what appears to be weighted data.
       While the estimated values of the parameters will always be calculated taking the weights into account,
       there are multiple ways to estimate the errors of the parameters. You are advised to make an 
       explicit choice for the error calculation:
           - Either provide SumW2Error(true), to calculate a sum-of-weights-corrected HESSE error matrix
             (error will be proportional to the number of events in MC).
           - Or provide SumW2Error(false), to return errors from original HESSE error matrix
             (which will be proportional to the sum of the weights, i.e., a dataset with <sum of weights> events).
           - Or provide AsymptoticError(true), to use the asymptotically correct 